# Basic statistics of SARS-CoV-2 virus
-   What is the size of SARS-CoV-2 genome?
-   What is the GC content of the genome?
    -   Is it different from other coronavirus?
-   How many genes in the SARS-CoV-2 genome?
    -   how many genes recorded in the NCBI page?
    -   Can it be predicted by programming? (by searching start codon ATC?)
-   How many functional proteins?
    -   What are there size?

##  What is the size of SARS-CoV-2 genome?

In [11]:
import sys
sys.path.append('../')
from Bio import Entrez, SeqIO
from conf import config
Entrez.email = config.email
# Search SARS-CoV-2 genome refseq using Entrez
handle = Entrez.esearch(db='nucleotide', term='"Severe acute respiratory syndrome coronavirus 2"[Organism] AND refseq[Filter]')
record = Entrez.read(handle)
record

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['1798174254'], 'TranslationSet': [{'From': '"Severe acute respiratory syndrome coronavirus 2"[Organism]', 'To': '"Severe acute respiratory syndrome coronavirus 2"[Organism]'}], 'TranslationStack': [{'Term': '"Severe acute respiratory syndrome coronavirus 2"[Organism]', 'Field': 'Organism', 'Count': '2588911', 'Explode': 'Y'}, {'Term': 'refseq[Filter]', 'Field': 'Filter', 'Count': '81198448', 'Explode': 'N'}, 'AND'], 'QueryTranslation': '"Severe acute respiratory syndrome coronavirus 2"[Organism] AND refseq[Filter]'}

In [13]:
# Fetch genbank record of SARS-CoV-2 referene genome
handle = Entrez.efetch(db='nucleotide', id=record['IdList'][0], rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()
print(record)

ID: NC_045512.2
Name: NC_045512
Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Database cross-references: BioProject:PRJNA485481
Number of features: 57
/molecule_type=ss-RNA
/topology=linear
/data_file_division=VRL
/date=18-JUL-2020
/accessions=['NC_045512']
/sequence_version=2
/keywords=['RefSeq']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
/references=[Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Programmed ribosomal frameshifting in decoding the SARS-CoV genome', ...), Reference(title='The structure of a rigorously conserved RNA element within the SARS virus genome', ...), Reference(title="A phyl

In [20]:
print('The size of SARS-CoV-2 reference genome is %d bp' % len(record))
print('There are %d features in the reference genome' % len(record.features))

The size of SARS-CoV-2 reference genome is 29903 bp
There are 57 features in the reference genome


In [23]:
for feature in filter(lambda x: x.type == 'CDS', record.features):
    print(feature)

type: CDS
location: join{[265:13468](+), [13467:21555](+)}
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:43740578']
    Key: gene, Value: ['ORF1ab']
    Key: locus_tag, Value: ['GU280_gp01']
    Key: note, Value: ['pp1ab; translated by -1 ribosomal frameshift']
    Key: product, Value: ['ORF1ab polyprotein']
    Key: protein_id, Value: ['YP_009724389.1']
    Key: ribosomal_slippage, Value: ['']
    Key: translation, Value: ['MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAF

##  What is the GC content of the genome?

In [24]:
from Bio.SeqUtils import GC
from collections import Counter
print('The GC content of SARS-CoV-2 genome is %d%%' % GC(record.seq))
genome_size = len(record)
nt_comp = Counter(record.seq)
print('Neucleotide composition of SARS-CoV-2: A(%d%%), T(%d%%), C(%d%%), G(%d%%)' % (nt_comp['A']/genome_size*100, nt_comp['T']/genome_size*100, nt_comp['C']/genome_size*100, nt_comp['G']/genome_size*100))

The GC content of SARS-CoV-2 genome is 37%
Neucleotide composition of SARS-CoV-2: A(29%), T(32%), C(18%), G(19%)


In [42]:
class Seq:
    def __init__(self, name, seqrecord):
        self._seq = seqrecord.seq
        self._name = name
        self._id = seqrecord.id
        self._size = None
        self._GC = None
    def getName(self):
        return self._name
    def getID(self):
        return self._id
    def reportSize(self):
        self._size = len(self._seq)
        print('The size of {} genome is {:,} bp'.format(self._name ,self._size))
    def genomeGC(self):
        self._GC = GC(self._seq)
        return self._GC
    def reportGC(self):
        self.genomeGC()
        print('The GC content of %s genome is %d%%' % (self._name, self._GC))
    def ntcomponent(self):
        d = Counter(self._seq)
        self._ntcomponents = {k: v/self._size*100 for (k, v) in d.items()}
        return self._ntcomponents
    def reportNtComponent(self):
        self.ntcomponent()
        print('Neucleotide composition of %s: A(%d%%), T(%d%%), C(%d%%), G(%d%%)' % (self._name, self._ntcomponents['A'], self._ntcomponents['T'], self._ntcomponents['C'], self._ntcomponents['G']))

def testGenomeInfo(filepath):
    record = Seq('SARS-CoV-2', next(SeqIO.parse(filepath, 'fasta')))
    record.reportSize()
    record.reportGC()
    record.reportNtComponent()

filepath = '../data/01_raw/NC_045512.fasta'
testGenomeInfo(filepath)

The size of SARS-CoV-2 genome is 29,903 bp
The GC content of SARS-CoV-2 genome is 37%
Neucleotide composition of SARS-CoV-2: A(29%), T(32%), C(18%), G(19%)


## Is it different from other coronavirus?
-   Feline infectious peritonitis virus (NC_002306.3)
-   Human coronavirus 229E (NC_002645.1)
-   Human coronavirus NL64 (NC_005831.2)
-   Miniopterus bat coronavirus 1A (NC_010437.1)
-   Miniopterus bat coronavirus 1B (EU420137.1)
-   Miniopterus bat coronavirus HKU8 (NC_010438.1)
-   Porcine epidemic diarrhea virus (NC_003436.1)
-   Porcine respiratory coronavirus (DQ811787.1)
-   Rhinolophus bat coronavirus HKU2 (NC_009988.1)
-   Scotophilus bat coronavirus 512 (NC_009657.1)
-   Transmissible gastroenteritis virus (NC_038861.1)
-   Bovine coronavirus (NC_003045.1)
-   Equine coronavirus (LC061274.1)
-   Human coronavirus HKU1 (NC_006577.2)
-   Human coronavirus OC43 (NC_006213.1)
-   Mouse hepatitis virus (NC_001846.1)
-   Porcine hemagglutinating encephalomyelitis virus (DQ011855.1)
-   Severe acute respiratory syndrome-related coronavirus 2	(NC_045512.2)
-   Severe acute respiratory syndrome-related coronavirus	(NC_004718.3)
-   SARS-related Rhinolophus bat coronavirus HKU3/	(NC_009694.1)
-   Middle East respiratory syndrome-related coronavirus	(NC_019843.3)
-   Bat coronavirus HKU9-1	(NC_009021.1)
-   Pipistrellus bat coronavirus HKU5	(NC_009020.1)
-   Tylonycteris bat coronavirus HKU4	(NC_009019.1)
-   Avian infectious bronchitis virus	(NC_001451.1)
-   Beluga whale coronavirus SW1	(NC_010646.1)
-   Turkey coronavirus	(NC_010800.1)
-   Bulbul coronavirus HKU11-934	(NC_011547.1)
-   Munia coronavirus HKU13-3514	(NC_011550.1)
-   Thrush coronavirus HKU12-600	(NC_011549.1)

In [41]:
import os
import gzip
import numpy as np
DIR= r'../data/01_raw/refseq/viral/'
viral_records = []
for folder in os.listdir(DIR):
    for f in os.listdir(os.path.join(DIR, folder)):
        if f.endswith(".fna.gz"):
            with gzip.open(os.path.join(DIR, folder, f), 'rt') as handle:
                viral_record = SeqIO.parse(handle, 'fasta')
                for viral in viral_record:
                    viral_records.append(viral)

ls_corona = ['NC_002306.3', 'NC_002645.1', 'NC_005831.2', 'NC_010437.1', 'EU420137.1',
'NC_010438.1', 'NC_003436.1', 'DQ811787.1', 'NC_009988.1', 'NC_009657.1', 'NC_038861.1', 
'NC_003045.1', 'LC061274.1', 'NC_006577.2', 'NC_006213.1', 'NC_001846.1', 'DQ011855.1', 
'NC_004718.3', 'NC_009694.1', 'NC_019843.3', 'NC_009021.1', 'NC_009020.1', 'NC_009019.1', 
'NC_001451.1', 'NC_010646.1', 'NC_010800.1', 'NC_011547.1', 'NC_011550.1', 'NC_011549.1']
GCs_viral = []
for viral in viral_records:
    if viral.id in ls_corona:
        GC_viral = GC(viral.seq)
        GCs_viral.append(GC_viral)

    else:
        continue
print('The mean GC content of coronavirus genome is %d%%' % np.mean(GCs_viral))


The mean GC content of coronavirus genome is 39%


In [40]:
import os
import gzip
import numpy as np
class ViralSeqs:
    def __init__(self, name, lst_accessions):
        self._name = name
        self._viralseqs = []
        self.filterViral(lst_accessions)
    def getViralseqs(self):
        return self._viralseqs
    def getNumberofSeqs(self):
        self._num = len(self._viralseqs)
    def filterViral(self, lst_accessions):
        DIR= r'../data/01_raw/refseq/viral/'
        for folder in os.listdir(DIR):
            for f in os.listdir(os.path.join(DIR, folder)):
                if f.endswith(".fna.gz"):
                    with gzip.open(os.path.join(DIR, folder, f), 'rt') as handle:
                        viral_record = SeqIO.parse(handle, 'fasta')
                        for viral in viral_record:
                            if viral.id.split('.')[0] in lst_accessions:
                                self._viralseqs.append(viral)
        return self._viralseqs
    def getName(self):
        return self._name
    def meanViralGC(self):
        self._meanGC = np.mean([GC(s.seq) for s in self._viralseqs])
        return self._meanGC
    def reportmeanGC(self):
        self.meanViralGC()
        print('The mean GC content of %s genome is %.2f%%' % (self._name, self._meanGC))
    def meanntcomponent(self):
        ntcomponents = {'A':[], 'T':[], 'C': [], 'G': []}
        for viral in self._viralseqs:
            d = Counter(viral.seq.upper())
            ntcomp = {k: v/len(viral.seq)*100 for (k, v) in d.items()}
            ntcomponents['A'].append(ntcomp['A'])
            ntcomponents['T'].append(ntcomp['T'])
            ntcomponents['C'].append(ntcomp['C'])
            ntcomponents['G'].append(ntcomp['G'])
        self._meancomp = {k: np.mean(v) for (k, v) in ntcomponents.items()}
        return self._meancomp
    def reportmeancomp(self):
        self.meanntcomponent()
        print('The mean nucleotide composition of %s genome: A(%.2f%%), T(%.2f%%), C(%.2f%%), G(%.2f%%)' % (self._name, self._meancomp['A'], self._meancomp['T'], self._meancomp['C'], self._meancomp['G']))
    def reportnumofseqs(self):
        self.getNumberofSeqs()
        print('%s contains %d genome sequences' % (self._name, self._num))   

def testViralSeqs():
    acc_corona =  ['NC_002306', 'NC_002645', 'NC_005831', 'NC_010437', 'EU420137',
    'NC_010438', 'NC_003436', 'DQ811787', 'NC_009988', 'NC_009657', 'NC_038861', 
    'NC_003045', 'LC061274', 'NC_006577', 'NC_006213', 'NC_001846', 'DQ011855', 
    'NC_004718', 'NC_009694', 'NC_019843', 'NC_009021', 'NC_009020', 'NC_009019', 
    'NC_001451', 'NC_010646', 'NC_010800', 'NC_011547', 'NC_011550', 'NC_011549']
    coronavirus = ViralSeqs('Coronavirus', acc_corona)
    coronavirus.reportnumofseqs()
    coronavirus.reportmeanGC()
    coronavirus.reportmeancomp()

    
testViralSeqs()
    

Coronavirus contains 24 genome sequences
The mean GC content of Coronavirus genome is 39.01%
The mean nucleotide composition of Coronavirus genome: A(27.22%), T(33.77%), C(17.57%), G(21.44%)


#   How many genes in the SARS-CoV-2 genome?  
-   identify open reading frame in the SARS-CoV-2 genome.

In [25]:
from Bio.Data import CodonTable

class ORF():
    def __init__(self, strand, frame, startcodon, stopcodon, seq) -> None:
        self.strand = strand
        self.frame = frame
        self.startcodon = startcodon
        self.stopcodon = stopcodon
        self.seq = seq


class ORFfinder:
    def __init__(self):
        self._codontable = CodonTable.standard_dna_table
    
    def findorf(self, seq):
        self._orfs = []
        for strand in ['+', '-']:
            if strand == '-':
                seq = seq.reverse_complement()
            for frame in range(3):
                first_startcodon_idx = None
                for i in range(frame, len(str(seq)), 3):
                    codon = str(seq)[i: i + 3]
                    if not first_startcodon_idx and codon == 'ATG':
                        first_startcodon_idx = i
                    if first_startcodon_idx and codon in self._codontable.stop_codons:
                        orf = str(seq)[first_startcodon_idx:i+3]
                        if (i - first_startcodon_idx) >= 300:
                            if strand == '+':
                                self._orfs.append(ORF(strand, frame + 1, first_startcodon_idx + 1, i + 3, orf))
                            else:
                                self._orfs.append(ORF(strand, frame + 1, len(str(seq)) - (first_startcodon_idx + 1) + 1, len(str(seq)) - (i + 3) + 1 , orf))
                        first_startcodon_idx = None
        return self._orfs
                       
    def reportorf(self, seq):
        self.findorf(seq)
        print('The Open Reading Frame:')
        print('#\tstrand\tframe\tstart\tstop\tlength')
        for i, orf in enumerate(self._orfs):
            print('%d\t%s\t%d\t%d\t%d\t%d' % (i + 1, orf.strand, orf.frame ,orf.startcodon, orf.stopcodon, abs(orf.stopcodon - orf.startcodon) + 1))

    


def testorffinder(seq):
    orffinder = ORFfinder()
    orffinder.reportorf(seq)

testorffinder(record.seq)
        

The Open Reading Frame:
#	strand	frame	start	stop	length
1	+	1	13768	21555	7788
2	+	1	25393	26220	828
3	+	1	27394	27759	366
4	+	2	266	13483	13218
5	+	2	21536	25384	3849
6	+	2	28274	29533	1260
7	+	3	26523	27191	669
8	+	3	27894	28259	366
9	-	3	6489	6187	303
